In [18]:
import json
import os
import requests
from dotenv import load_dotenv
from time import time
# quitar los warnigngs de las peticiones
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [19]:
# leer el archivo .env
load_dotenv()

True

In [20]:
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")

In [21]:
def get_processor(url_nifi_api: str, processor_id: str, token=None, verify=False):
    """
    Obtiene y devuelve un procesador específico.

    Utiliza la API REST `/processors/{processor_id}`.

    :param url_nifi_api: Cadena de texto
    :param processor_id: Cadena de texto
    :param token: Token de acceso JWT
    :param verify: si se debe verificar el certificado SSL.
    :returns: Objeto JSON del procesador
    """

    # Cabecera de autorización
    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(token),
    }

    # Solicita el procesador y lo convierte a JSON
    response = requests.get(url_nifi_api + f"processors/{processor_id}", headers=header, verify=verify)
    return json.loads(response.content)

In [22]:
def update_processor_status(processor_id: str, new_state: str, token, url_nifi_api, verify=False):
    """
    Inicia o detiene un procesador recuperando su información
    para obtener la revisión actual y luego enviando un JSON con el estado deseado.

    :param processor_id: ID del procesador al que se le aplicará el nuevo estado.
    :param new_state: Cadena de texto con el nuevo estado, valores aceptados:
                      STOPPED o RUNNING.
    :param token: Token JWT de acceso para NiFi.
    :param url_nifi_api: URL de la API de NiFi
    :param verify: si se debe verificar el certificado SSL.
    :return: None
    """

    # Obtener información del procesador desde `/processors/{processor_id}`
    processor = get_processor(url_nifi_api, processor_id, token)

    # Crear JSON con el nuevo estado y la revisión del procesador
    put_dict = {
        "revision": processor["revision"],
        "state": new_state,
        "disconnectedNodeAcknowledged": True,
    }

    # Enviar el JSON usando PUT
    payload = json.dumps(put_dict).encode("utf8")
    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(token),
    }
    response = requests.put(
        url_nifi_api + f"processors/{processor_id}/run-status",
        headers=header,
        data=payload,
        verify=verify
    )
    return response

In [23]:
def get_token(url_nifi_api: str, access_payload: dict, verify=False) -> str:
    """
    Obtiene un token JWT autenticando al usuario,
    usando la API REST `/access/token`.

    :param url_nifi_api: URL base de la API de NiFi.
    :param access_payload: diccionario con claves 'username' y 'password'
                           y sus respectivos valores.
    :param verify: si se debe verificar el certificado SSL.
    :return: Token JWT
    """

    header = {
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "*/*",
    }
    response = requests.post(
        url_nifi_api + "access/token", headers=header, data=access_payload, verify=verify
    )
    return response.content.decode("ascii")

In [24]:
access_payload = {
        "username": USER,
        "password": PASSWORD,
    }  # e.g. retrieve via Airflow's `BaseHook` functionality
url_nifi_api = "https://192.9.200.15:8443/nifi-api/"  # e.g. retrieve via Airflow's `BaseHook` functionality
token = get_token(url_nifi_api, access_payload, verify=False)

In [34]:
id_processor = '685de29a-08d3-31f0-5699-a2553e6f2501'

In [37]:
running_processor = update_processor_status(id_processor, "RUN_ONCE", token, url_nifi_api, verify=False)